# Description
Immuno biomarkers analysis for samples with COVID-19 in different stages and Down syndrome.

## 1. Update data with COVID and DS columns. Add SImAge and IPAge values and accelerations.
## 2. COVID-19 analysis:
- Non-longitudinal analysis:
    - Biomarkers and SImAge acceleration distributions in different COVID-19 groups.
    - Correlation between biomarkers and SImAge acceleration.
- Longitudinal analysis:
    - Trajectories with time points.
    - Repeated measures tests.
## 3. Down syndrome analysis:
- Biomarkers and SImAge acceleration distributions in different groups.

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
import seaborn as sns
import plotly.express as px
import statsmodels.formula.api as smf
import plotly.graph_objects as go
from scripts.python.routines.manifest import get_manifest
from scripts.python.routines.plot.save import save_figure
from scripts.python.routines.plot.layout import add_layout
from statsmodels.stats.multitest import multipletests
import plotly.io as pio
pio.kaleido.scope.mathjax = None
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=False)
from scipy.stats import mannwhitneyu, median_test, kruskal
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.patheffects as path_effects
import random
import pathlib
from tqdm import tqdm
from src.utils.plot.bioinfokit import mhat, volcano
import gseapy as gp
import mygene
from sklearn.decomposition import PCA, IncrementalPCA, KernelPCA, TruncatedSVD
from sklearn.decomposition import MiniBatchDictionaryLearning, FastICA
from sklearn.random_projection import GaussianRandomProjection, SparseRandomProjection
from sklearn.manifold import MDS, Isomap, TSNE, LocallyLinearEmbedding
import upsetplot
from matplotlib_venn import venn2, venn2_circles
from itertools import chain
from sklearn.metrics import mean_absolute_error
from scripts.python.routines.plot.colorscales import get_continuous_color
import plotly
from scripts.python.routines.plot.p_value import add_p_value_annotation
from scripts.python.routines.sections import get_sections
from statannotations.Annotator import Annotator
import functools
import matplotlib.lines as mlines


def conjunction(conditions):
    return functools.reduce(np.logical_and, conditions)


def disjunction(conditions):
    return functools.reduce(np.logical_or, conditions)

# Init data

In [ ]:
path = f"D:/YandexDisk/Work/pydnameth/datasets/GPL21145/GSEUNN"
df = pd.read_excel(f"{path}/data/immuno/df_samples(all_1052_121222)_proc(raw)_imp(fast_knn)_replace(quarter).xlsx", index_col=0)
feats = pd.read_excel(f"{path}/data/immuno/feats_con.xlsx", index_col=0).index.values
path_save = f"{path}/special/052_immuno_covid_down_syndrome"
pathlib.Path(f"{path_save}").mkdir(parents=True, exist_ok=True)

## 1. Update data with COVID and DS columns. Add SImAge and IPAge values and accelerations.

In [ ]:
df_covid_ds = pd.read_excel(f"{path}/data/immuno/update_COVID_DownSyndrome.xlsx", index_col=0)
index_diff_base_covid_ds = df.index.difference(df_covid_ds.index)
print(f"index_diff_base_covid_ds: {index_diff_base_covid_ds}")
index_diff_covid_ds_base = df_covid_ds.index.difference(df.index)
print(f"index_diff_covid_ds_base: {index_diff_covid_ds_base}")

cols = {
    'COVID-19 stage': 'Сovid stage',
    'COVID-19 sample ID': 'ID Covid',
    'Down syndrome status': 'Relation to Down Syndrome'
}
for new_col, old_col in cols.items():
    df.loc[df.index.values, new_col] = df_covid_ds.loc[df.index.values, old_col]

df_simage = pd.read_excel(f"{path}/data/immuno/update_SImAge.xlsx", index_col=0)
index_diff_base_simage = df.index.difference(df_simage.index)
print(f"index_diff_base_simage: {index_diff_base_simage}")
index_diff_simage_base = df_simage.index.difference(df.index)
print(f"index_diff_simage_base: {index_diff_simage_base}")

cols = {
    'SImAge': 'Prediction',
    'SImAge acceleration': 'Prediction error',
    '|SImAge acceleration|': 'Prediction error abs'
}
for new_col, old_col in cols.items():
    df.loc[df.index.values, new_col] = df_simage.loc[df.index.values, old_col]

df.to_excel(f"{path}/data/immuno/df.xlsx", index_label='index')

## 2. COVID-19 analysis:
- Non-longitudinal analysis:
    - Biomarkers and SImAge acceleration distributions in different COVID-19 groups.
    - Correlation between biomarkers and SImAge acceleration.
- Longitudinal analysis:
    - Trajectories with time points.
    - Repeated measures tests.

### Plot count and age distributions

In [ ]:
path_curr = f"{path_save}/01_COVID19"
pathlib.Path(f"{path_curr}").mkdir(parents=True, exist_ok=True)

colors = {
    'Acute': 'crimson',
    'Dynamics': 'gold',
    'Reconvalescent': 'cyan',
    'Reconvalescent after 1 year': 'lime'
}

df_covid = df.loc[df['COVID-19 stage'].isin(list(colors.keys())), :]
df_covid.to_excel(f"{path_curr}/df_covid.xlsx", index_label='index')

fig = plt.figure(figsize=(3, 4))
sns.set_theme(style='whitegrid', font_scale=1)
countplot = sns.countplot(
    data=df_covid,
    y='COVID-19 stage',
    edgecolor='black',
    palette=colors,
    orient='v',
    order=list(colors.keys())
)
countplot.bar_label(countplot.containers[0])
countplot.set_xlabel("Count")
countplot.set_ylabel("")
countplot.set_title(f"")
plt.savefig(f"{path_curr}/countplot.png", bbox_inches='tight', dpi=400)
plt.savefig(f"{path_curr}/countplot.pdf", bbox_inches='tight')
plt.close(fig)

hist_bins = np.linspace(5, 115, 23)
fig = plt.figure(figsize=(6, 4))
sns.set_theme(style='whitegrid')
histplot = sns.histplot(
    data=df_covid,
    hue_order=list(colors.keys())[::-1],
    bins=hist_bins,
    x="Age",
    hue="COVID-19 stage",
    edgecolor='black',
    palette=colors,
    multiple="stack"
)
sns.move_legend(
    histplot, "lower center",
    bbox_to_anchor=(.5, 1), ncol=4, title="COVID-19 stage", frameon=True,
)
plt.setp(histplot.get_legend().get_texts(), fontsize='7') # for legend text
plt.setp(histplot.get_legend().get_title(), fontsize='10')
plt.savefig(f"{path_curr}/histplot.png", bbox_inches='tight', dpi=400)
plt.savefig(f"{path_curr}/histplot.pdf", bbox_inches='tight')
plt.close(fig)

df_rep_meas = df_covid['COVID-19 sample ID'].value_counts().to_frame()
df_rep_meas.rename(columns={'COVID-19 sample ID': 'Time points'}, inplace=True)
df_rep_meas.to_excel(f"{path_curr}/df_rep_meas.xlsx", index_label='index')

samples_mtp = df_rep_meas.index[df_rep_meas['Time points'] != 1].values # 'mtp' means multiple time points
df_covid_rep_meas = df_covid.loc[df_covid['COVID-19 sample ID'].isin(samples_mtp), :]
df_covid_rep_meas.to_excel(f"{path_curr}/df_covid_rep_meas.xlsx", index_label='index')

fig = plt.figure(figsize=(3, 4))
sns.set_theme(style='whitegrid', font_scale=1)
countplot = sns.countplot(
    data=df_rep_meas,
    y='Time points',
    edgecolor='black',
    orient='v',
)
countplot.bar_label(countplot.containers[0])
countplot.set_xlabel("Count")
countplot.set_ylabel("")
countplot.set_title(f"")
plt.savefig(f"{path_curr}/rep_meas.png", bbox_inches='tight', dpi=400)
plt.savefig(f"{path_curr}/rep_meas.pdf", bbox_inches='tight')
plt.close(fig)

### Non-longitudinal analysis: Statistical tests for immunology biomarkers and age accelerations

In [ ]:
path_curr = f"{path_save}/01_COVID19/non_longitudinal"
pathlib.Path(f"{path_curr}/feats").mkdir(parents=True, exist_ok=True)

df_stat = pd.DataFrame(index=list(feats))

for feat in feats:
    vals = {}
    for group in colors.keys():
        vals[group] = df_covid.loc[df_covid['COVID-19 stage'] == group, feat].values
        df_stat.at[feat, f"mean_{group}"] = np.mean(vals[group])
        df_stat.at[feat, f"median_{group}"] = np.median(vals[group])
        df_stat.at[feat, f"q75_{group}"], df_stat.at[feat, f"q25_{group}"] = np.percentile(vals[group], [75 , 25])
        df_stat.at[feat, f"iqr_{group}"] = df_stat.at[feat, f"q75_{group}"] - df_stat.at[feat, f"q25_{group}"]
    _, df_stat.at[feat, "kw_pval"] = kruskal(*vals.values())
    _, df_stat.at[feat, "mw_01_pval"] = mannwhitneyu(vals['Acute'], vals['Dynamics'], alternative='two-sided')
    _, df_stat.at[feat, "mw_12_pval"] = mannwhitneyu(vals['Dynamics'], vals['Reconvalescent'], alternative='two-sided')
    _, df_stat.at[feat, "mw_23_pval"] = mannwhitneyu(vals['Reconvalescent'], vals['Reconvalescent after 1 year'], alternative='two-sided')
    _, df_stat.at[feat, "mw_03_pval"] = mannwhitneyu(vals['Acute'], vals['Reconvalescent after 1 year'], alternative='two-sided')

_, df_stat["kw_pval_fdr_bh"], _, _ = multipletests(df_stat["kw_pval"], 0.05, method='fdr_bh')
_, df_stat["mw_01_pval_fdr_bh"], _, _ = multipletests(df_stat["mw_01_pval"], 0.05, method='fdr_bh')
_, df_stat["mw_12_pval_fdr_bh"], _, _ = multipletests(df_stat["mw_12_pval"], 0.05, method='fdr_bh')
_, df_stat["mw_23_pval_fdr_bh"], _, _ = multipletests(df_stat["mw_23_pval"], 0.05, method='fdr_bh')
_, df_stat["mw_03_pval_fdr_bh"], _, _ = multipletests(df_stat["mw_03_pval"], 0.05, method='fdr_bh')

df_stat.sort_values([f"kw_pval_fdr_bh"], ascending=[True], inplace=True)

vals = {}
for group in colors.keys():
    vals[group] = df_covid.loc[df_covid['COVID-19 stage'] == group, 'SImAge acceleration'].values
    df_stat.at['SImAge acceleration', f"mean_{group}"] = np.mean(vals[group])
    df_stat.at['SImAge acceleration', f"median_{group}"] = np.median(vals[group])
    df_stat.at['SImAge acceleration', f"q75_{group}"], df_stat.at['SImAge acceleration', f"q25_{group}"] = np.percentile(vals[group], [75 , 25])
    df_stat.at['SImAge acceleration', f"iqr_{group}"] = df_stat.at['SImAge acceleration', f"q75_{group}"] - df_stat.at['SImAge acceleration', f"q25_{group}"]
_, df_stat.at['SImAge acceleration', "kw_pval"] = kruskal(*vals.values())
_, df_stat.at['SImAge acceleration', "mw_01_pval"] = mannwhitneyu(vals['Acute'], vals['Dynamics'], alternative='two-sided')
_, df_stat.at['SImAge acceleration', "mw_12_pval"] = mannwhitneyu(vals['Dynamics'], vals['Reconvalescent'], alternative='two-sided')
_, df_stat.at['SImAge acceleration', "mw_23_pval"] = mannwhitneyu(vals['Reconvalescent'], vals['Reconvalescent after 1 year'], alternative='two-sided')
_, df_stat.at['SImAge acceleration', "mw_03_pval"] = mannwhitneyu(vals['Acute'], vals['Reconvalescent after 1 year'], alternative='two-sided')

df_stat.to_excel(f"{path_curr}/stat.xlsx", index_label='Features')

for feat_id, (feat, row) in enumerate(df_stat.iterrows()):
    plt.figure(figsize=(8, 4))
    sns.set_theme(style='whitegrid')
    violin = sns.violinplot(
        data=df_covid,
        x='COVID-19 stage',
        y=feat,
        palette=colors,
        scale='width',
        order=list(colors.keys()),
        saturation=0.75,
    )
    violin.set_ylabel(feat)
    if feat != 'SImAge acceleration':
        violin.set_xlabel(f"Kruskal-Wallis p-value: {row['kw_pval_fdr_bh']:.2e}")
        mw_01_pval = row["mw_01_pval_fdr_bh"]
        mw_12_pval = row["mw_12_pval_fdr_bh"]
        mw_23_pval = row["mw_23_pval_fdr_bh"]
        mw_03_pval = row["mw_03_pval_fdr_bh"]
    else:
        violin.set_xlabel(f"Kruskal-Wallis p-value: {row['kw_pval']:.2e}")
        mw_01_pval = row["mw_01_pval"]
        mw_12_pval = row["mw_12_pval"]
        mw_23_pval = row["mw_23_pval"]
        mw_03_pval = row["mw_03_pval"]

    pval_formatted = [f'{mw_01_pval:.2e}', f'{mw_12_pval:.2e}', f'{mw_23_pval:.2e}', f'{mw_03_pval:.2e}']
    annotator = Annotator(
        violin,
        pairs=[('Acute', 'Dynamics'), ('Dynamics', 'Reconvalescent'), ('Reconvalescent', 'Reconvalescent after 1 year'), ('Acute', 'Reconvalescent after 1 year')],
        data=df,
        x='COVID-19 stage',
        y=feat,
        order=list(colors.keys())
    )
    annotator.set_custom_annotations(pval_formatted)
    annotator.configure(loc='outside')
    annotator.annotate()
    if feat != 'SImAge acceleration':
        plt.savefig(f"{path_curr}/feats/{feat_id}_{feat}.png", bbox_inches='tight', dpi=400)
        plt.savefig(f"{path_curr}/feats/{feat_id}_{feat}.pdf", bbox_inches='tight')
    else:
        plt.savefig(f"{path_curr}/{feat}.png", bbox_inches='tight', dpi=400)
        plt.savefig(f"{path_curr}/{feat}.pdf", bbox_inches='tight')
    plt.close()